Step 1: Install Required Libraries

In [1]:
pip install -U langchain langchain-google-genai faiss-cpu streamlit


In [2]:
pip install -U langchain-community


In [3]:
!pip install transformers sentence-transformers faiss-cpu streamlit

In [4]:
pip install -qU langchain-text-splitters langchain-core


In [5]:
pip install langchain-huggingface transformers

In [6]:
pip install langchain-classic

**Load and Vectorize Documents**

In [ ]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from google import genai

os.environ["GOOGLE_API_KEY"] = "use api"

# Load documents
documents = []
for file in os.listdir("/content/data"): #your dataset path
    if file.endswith(".txt"):
        loader = TextLoader(f"data/{file}")
        documents.extend(loader.load())

# Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
docs = text_splitter.split_documents(documents)

# Create embeddings
# Using free HuggingFace embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save index
vectorstore.save_local("faiss_index")


**Step 4: Initialize Gemini LLM**

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_output_tokens=512
)

**Step 5: Create Optimized Context-Aware Prompt**

In [16]:
from langchain_core.prompts import PromptTemplate

QA_PROMPT = PromptTemplate.from_template("""
You are a context-aware academic assistant.

Your role is to answer user questions using:
1. The provided external knowledge context (retrieved documents)
2. The ongoing conversation history

Behavior rules:
- Always prioritize the retrieved context when answering.
- Use conversation history to correctly interpret follow-up questions.
- Do NOT repeat definitions unless explicitly requested.
- Answer only what is asked; do not add unrelated information.
- Be concise, factual, and well-structured.
- Use bullet points or numbered lists when listing items.
- If the answer is not present in the provided context, clearly say:
  "The information is not available in the provided documents."
- Do not hallucinate or assume missing facts.
- Maintain an academic, neutral tone suitable for students.

Response style:
- Definitions: 1–2 clear sentences.
- Lists: Short bullet points.
- Follow-up questions: Use prior context to avoid repetition.
- No emojis, no informal language, no verbosity.

You are part of a Retrieval-Augmented Generation (RAG) system.


Context:
{context}

Chat History:
{chat_history}

Question:
{question}

Answer:
""")


**Step 6: Build Conversational RAG Chain**

In [17]:
from langchain_classic.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=False,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT}
)
print("RAG chatbot is ready.")

RAG chatbot is ready.


In [19]:
# ----------------------------
# Initialize chat history (ONLY ONCE)
# ----------------------------
chat_history = []

# ----------------------------
# Step 1: First Question
# ----------------------------
question_1 = "What is Natural Language Processing (NLP)?"

response1 = qa_chain.invoke({
    "question": question_1,
    "chat_history": chat_history
})

answer_1 = response1.get("answer", "I don't know.")

print("AI:", answer_1)

# Store conversation properly
chat_history.append((question_1, answer_1))


# ----------------------------
# Step 2: Context-Aware Follow-Up
# ----------------------------
question_2 = "How it is related to Artificial Intelligence?"

response2 = qa_chain.invoke({
    "question": question_2,
    "chat_history": chat_history
})

answer_2 = response2.get("answer", "I don't know.")

print("AI:", answer_2)

# Store follow-up interaction
chat_history.append((question_2, answer_2))


AI: Natural Language Processing (NLP) is the processing of natural language information by a computer. It is a subfield of computer science closely associated with artificial intelligence.
AI: Natural Language Processing (NLP) is a subfield of computer science and is closely associated with artificial intelligence.
